### import and read file

In [46]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
feature = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
"num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
"num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
"rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
"dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
"dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
train_data = pd.read_csv('./KDDTrain+.txt' , names = feature)
test_data = pd.read_csv('./KDDTest+.txt' , names = feature)
trainlen = len(train_data)
testlen = len(test_data)


### 预处理
读入数据后使用info函数对数据进行初步检查,无缺失情况。
将训练与测试数据暂时合并，进行预处理。步骤如下：
+ 将不需要的difficulty列删去
+ 根据training_attack_types.txt中的信息修改标签
+ 将数值数据z-score标准化
+ 将非数值的维度通过one hot encode数值化（protocol_type, service, flag）

做完处理后进行数据分离

In [44]:
def change_label(df):
    f = open('./training_attack_types.txt' , 'r')
    change_list = [i[:-1].split(' ') for i in f.readlines()]
    for i in change_list:
        df.label.replace([i[0]] , [i[1]] , inplace = True)
def numbric(df):
    df = pd.get_dummies(df, columns = ['protocol_type' , 'service' , 'flag'] , prefix = '', prefix_sep = '')
    return df
def standardization(df , col):

    std_scaler = StandardScaler()
    for i in col:
        arr = np.array(df[i])
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr) , 1))
    return df

def pre_proc(df):
    df.drop(['difficulty'] , axis=1 , inplace=True)
    change_label(df)

    num_col = [feature[0]] + feature[4:-2]
    df = standardization(df , num_col)
    df = numbric(df)
    return df

In [ ]:
temp_train = pd.DataFrame({'Istrain':[1]*trainlen})
temp_test = pd.DataFrame({'Istrain':[0]*testlen})
temptrain = pd.concat([train_data , temp_train] , axis = 1)
temptest = pd.concat([test_data , temp_test] , axis = 1)
data = pd.concat([temptrain , temptest] , ignore_index=True)

data = pre_proc(data)
data = data.groupby('Istrain')
train_data = data.get_group(1)
test_data = data.get_group(0)
test_data.rename(lambda s:s-trainlen , inplace = True)


### 训练
使用K-means算法进行聚类，在进入算法前将label信息暂时删去，在算法得到聚类中心后，对train数据进行一次分类，并用每一个中心得到结果最多的结果作为该中心的类标，得到中心与类标的映射，实现分类器功能

In [80]:
#train
n_clusters = 5
temp_train = train_data.drop(['label'] , axis = 1 , inplace=False)
km = KMeans(n_clusters=n_clusters , init='k-means++' , max_iter=400).fit(temp_train)


In [ ]:
km_center = km.cluster_centers_
km_label = km.labels_
label_stat = [[0]*5 for i in range(n_clusters)]
labels = ['normal','dos' , 'probe' , 'r2l' , 'u2r'] 
for i in range(len(km_label)):
    l = train_data['label'][i]
    l_ind = labels.index(l)
    label_stat[km_label[i]][l_ind] += 1
label_dic = [i.index(max(i)) for i in label_stat]



### 测试
在测试集上测试效果，并计算正确率

In [84]:
temp_test = test_data.drop(['label'] , axis = 1 , inplace=False)
guess_label = km.predict(temp_test)

guess = 0

for i in range(len(guess_label)):
    l = test_data['label'][i]
    l_ind = labels.index(l)
    if label_dic[guess_label[i]] == l_ind:
        guess += 1
print('success:', guess)
print('total:' , len(guess_label))
print('success rate',guess / len(guess_label))

success: 15027
total: 22544
success rate 0.6665631653655074
